In [63]:
# Licensed under a 3-clause BSD style license - see LICENSE.rst
"""Session class driving the high level interface API"""
import logging
import yaml
import pandas as pd 
import json

from regions import CircleSkyRegion
from collections import defaultdict
from pathlib import Path
from typing import List


from astropy.units import Quantity
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table

# from pydantic import BaseModel
from pydantic.utils import deep_update

from gammapy.utils.units import energy_unit_format
from gammapy.utils.pbar import progress_bar
from gammapy.utils.scripts import make_path, read_yaml

from gammapy.datasets import (
    Datasets,  
    MapDataset, 
    FluxPointsDataset, 
    SpectrumDatasetOnOff, 
    SpectrumDataset,
)

from gammapy.estimators import FluxPoints, SensitivityEstimator
from gammapy.maps import Map, MapAxis, RegionGeom, WcsGeom
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    SkyModel, 
    Models,
    Model,
    DatasetModels, 
    FoVBackgroundModel, 
    Models, 
    SkyModel, 
    ExpCutoffPowerLawSpectralModel
)

from gammapy.data import DataStore
from gammapy.estimators import (
    ExcessMapEstimator,
    FluxPointsEstimator,
    LightCurveEstimator,
)
from gammapy.makers import (
    FoVBackgroundMaker,
    MapDatasetMaker,
    ReflectedRegionsBackgroundMaker,
    RingBackgroundMaker,
    SafeMaskMaker,
    SpectrumDatasetMaker,
)

from gammapy.data import Observation

from gammapy.stats import WStatCountsStatistic

from feupy.utils.string_handling import name_to_txt
from feupy.utils.datasets import cut_energy_table_fp, write_datasets, read_datasets

# from feupy.analysis import CounterpartsAnalysisConfig, SimulationConfig, CTAObservationAnalysisConfig
from feupy.cta.irfs import Irfs
from feupy.utils.coordinates import skcoord_to_dict, dict_to_skcoord

from feupy.analysis.config import CounterpartsConfig, SimulationConfig

from feupy.plotters import *

from feupy.catalog.config import *

from feupy.roi import ROI
from feupy.target import Target

from gammapy.data import FixedPointingInfo, PointingMode

from astropy.io.fits.verify import VerifyWarning
import warnings

/home/born-again/anaconda3/envs/gammapy-1.1/lib/python3.9/site-packages/pydantic/_migration.py:290: UserWarning: `pydantic.utils:deep_update` has been removed. We are importing from `pydantic.v1.utils:deep_update` instead.See the migration guide for more details: https://docs.pydantic.dev/latest/migration/
  warnings.warn(


In [64]:
__all__ = ["Counterparts", "Simulation"]

In [103]:
log = logging.getLogger(__name__)

class Counterparts:
    """Config-driven high level simulation interface.

    It is initialized by default with a set of configuration parameters and values declared in
    an internal high level interface model, though the user can also provide configuration
    parameters passed as a nested dictionary at the moment of instantiation. In that case these
    parameters will overwrite the default values of those present in the configuration file.

    Parameters
    ----------
    config : dict or `CounterpartsConfig`
        Configuration options following `CounterpartsConfig` schema
    """

    def __init__(self, config):
        self.config = config
        self.config.set_logging()
        self.roi = self._create_roi()
        self.target = None
        self.catalogs = None
        self.datasets = None
        self.sources = None
#         self.models = None
        self.pulsars = None
        self.dict_roi = None
        self.df_roi = None
        
#         self._obs = self.config.observations
#         self._obs_params = self.config.observations.parameters
#         self._obs_target = self.config.observations.target
#         self._obs_irfs = self.config.observations.irfs
#         self._datasets = self.config.datasets

#         self.pointing_position = self._create_pointing_position(
#             dict_to_skcoord(self._obs_target.position), 
#             self._obs_params.offset)
#         self.pointing = self._create_pointing(self.pointing_position)

#         self._ctao_perf = Irfs
#         self._ctao_perf.get_irfs(self._obs_irfs.opt)
#         self.observation = self._create_observation(
#             dict_to_skcoord(self._obs_target.position), 
#             self._obs_params.livetime, 
#             self._ctao_perf.irfs, 
#             self._ctao_perf.obs_loc
#         )
        
    
#         self.datastore = None
#         self.observations = None
#         self.datasets = None
#         self.fit = Fit()
#         self.fit_result = None
#         self.flux_points = None
#         self.dataset_map = None
#         self.pointing = dict_to_skcoord(self.config.target.position)
#         self._datasets_settings = self.config.datasets
#         self._observations_settings = self.config.observations
#         self._ctao_perf = Irfs
#         self._ctao_perf.get_irfs(self.config.irfs.opt)
#         self.observation = None
#         self.geom = None
#         self.energy_axis_true = self._make_energy_axis(self._datasets_settings.geom.axes.energy_true)
#         self.energy_axis_reco = self._make_energy_axis(self._datasets_settings.geom.axes.energy)
#         self.spectrum_dataset_empty = None
#         self.maker = None
#         self.safe_maker = None
#         self.spectrum_dataset = None
#         self.spectrum_dataset_onoff = None
#         self.wstat = None
        
#     @staticmethod
#     def _create_region_geometry(center, axes):
#         """Create the region geometry."""
#         on_lon = target_position.lon
#         on_lat = target_position.lat
#         frame = target_position.frame
#         pointing = SkyCoord(on_lon, on_lat, frame=frame)
#         self._create_pointing_position(
#             dict_to_skcoord(self._obs_target.position), 
#             self._obs_params.offset)
        
#         on_center = pointing.directional_offset_by(
#             position_angle=pointing.dec, 
#             separation=offset)
#         on_region = CircleSkyRegion(on_center, on_region_settings.radius)
#         return 


    def _create_target(self):
        """Create the target."""
        log.debug("Creating target.")
        target_settings = self.config.roi.target
        name = target_settings.name
        pos_ra = target_settings.position.lon
        pos_dec = target_settings.position.lat
        model = Model.from_dict(target_settings.model)
        return Target(name, pos_ra, pos_dec, spectral_model=model.spectral_model)
    
    def _create_roi(self):
        """Create the target."""
        log.debug("Creating target.")
        target = self._create_target()
        self.target = target
        on_region_radius = self.config.roi.region_radius
        return ROI(target, on_region_radius )
        
    @property
    def config(self):
        """Simulation configuration (`CounterpartsConfig`)"""
        return self._config

    @config.setter
    def config(self, value):
        if isinstance(value, dict):
            self._config = CounterpartsConfig(**value)
        elif isinstance(value, CounterpartsConfig):
            self._config = value
        else:
            raise TypeError("config must be dict or CounterpartsConfig.")

    @property
    def models(self):
        if not self.datasets:
            raise RuntimeError("No datasets defined. Impossible to set models.")
        return self.datasets.models

    @models.setter
    def models(self, models):
        self.set_models(models, extend=False)
        
        
    @staticmethod
    def _make_energy_axis(config_axis_energy, per_decade=True):
        return MapAxis.from_energy_bounds(        
            energy_min=config_axis_energy.min, 
            energy_max=config_axis_energy.max, 
            nbin=config_axis_energy.nbins, 
            per_decade=per_decade, 
            name=config_axis_energy.name,
            )
    
    def run(self):
        self._get_datasets()
        self._get_dict_roi()
        self._get_df_roi()
        
    def _get_datasets(self):
        """
        Select a catalog subset (only sources within a region of interest)
        """
        _catalogs = self.roi.catalogs
        self.pulsars = self.roi.pulsars
        e_ref_min = self.config.energy_range.min
        e_ref_max = self.config.energy_range.max

        
        datasets = Datasets() # global datasets object
        models = Models()  # global models object
        sources = [] # global sources object
        catalogs = []
        n_sources = 0 # number of sources
        n_flux_points = 0 # number of flux points tables
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', VerifyWarning)
            for catalog in _catalogs:
                indexes = []
                cat_tag = catalog.tag
                for source in catalog:
                    n_sources += 1   
                    source_name = source.name 
                    index = source.row_index
                    if cat_tag == PULSARTAG:
                        pass
                    else:
                        try:
                            flux_points = source.flux_points

                            spectral_model = source.spectral_model()
                            spectral_model_tag = spectral_model.tag[1]

                            if cat_tag == 'gamma-cat' or cat_tag == 'hgps':
                                dataset_name = f'{source_name}: {cat_tag}'
                            else: dataset_name = source_name

                            file_name = name_to_txt(dataset_name)

                            model = SkyModel(
                                name=f"{file_name}_{spectral_model_tag}",
                                spectral_model=spectral_model,
                                datasets_names=dataset_name
                            )

                            dataset = FluxPointsDataset(
                                models=model,
                                data=flux_points, 
                                name=dataset_name   
                            )

                            if any([e_ref_min !=  None, e_ref_max !=  None]):
                                dataset = cut_energy_table_fp(dataset, e_ref_min, e_ref_max) 

                            n_flux_points += 1
                            models.append(model)  # Add the model to models()
                            datasets.append(dataset)
                            sources.append(source)
                        except Exception as error:
                            indexes.append(index)
                            # By this way we can know about the type of error occurring
                            print(f'The error is: ({source_name}) {error}') 
                if len(indexes)>0:
                    if len(indexes)==1:
                        catalog.table.remove_row(indexes[0])
                    else: catalog.table.remove_rows(indexes)

                if len(catalog.table)>0:
                    catalogs.append(catalog)
            datasets.models = models
            self.datasets = datasets
#             self.models = models
            self.sources = sources
            self.catalogs = catalogs

            print(f"Total number of gamma sources: {len(self.sources)}")
            print(f"Total number of flux points tables: {n_flux_points}")
            print(f"Total number of pulsars: {len(self.pulsars)}")

#     def sensitivity_estimator(self):

             
    def _get_dict_roi(self):
        _dict_roi = {}

        roi_pos = self.roi.target.position 
        radius_roi = self.roi.radius 

        _sources = self.sources.copy()
        _sources.extend(self.pulsars)
        for index, source in enumerate(_sources):
            source_pos = source.position
            sep = source.position.separation(roi_pos).deg
            if index < len(self.datasets):
                name = self.datasets[index].name
            else: name = source.name
            _dict_roi[name] = {
                'position': source_pos,
                'separation':sep
            }

        self.dict_roi = _dict_roi
        
    def _get_df_roi(self):
        _dict = self.dict_roi

        df = pd.DataFrame()
        df["Source name"] = _dict.keys()
        df_ra = []
        df_dec = []
        df_sep = []

        for index, name in enumerate(_dict.keys()):
            df_ra.append(_dict[name]["position"].ra.deg)
            df_dec.append(_dict[name]["position"].dec.deg)
            df_sep.append(_dict[name]["separation"])

        df["RA(deg)"] = df_ra
        df["dec.(deg)"] = df_dec
        df["Sep.(deg)"] = df_sep
        self.df_roi = df
        
    def create_analysis_name(self): 
        """ ... """
        ss = f"{self.config.target.name}"
        ss += "_roi_{:.2f}".format(self.roi.radius).replace(' ', '')
        if e_ref_min is None: ss += ""
        else: ss += "_e_ref_min_{}".format(energy_unit_format(e_ref_min).replace(' ', ''))
        if e_ref_max is None: ss += ""
        else: ss += "_e_ref_max_{}".format(energy_unit_format(e_ref_max).replace(' ', ''))
        return ss
    
#     def create_analysis_path(self): 
#         """ ... """
#         return Path(f"analysis_counterparts/{self.create_analysis_name()}")

#     def write_datasets(self, overwrite=True, path_file=None):
#         """Write Datasets and Models to YAML file.

#             Parameters
#             ----------
#             overwrite : bool, optional
#                 Overwrite existing file. Default is True.  
#             """
        
#         if path_file is None:
#             path_file = Path(f"{self.create_analysis_path()}/datasets")
#         write_datasets(self.datasets, path_file, overwrite)
    
#     def read_datasets(self, path_file=None):
#         """Read Datasets and Models from YAML file."""

#         if path_file is None:
#             path_file = Path(f"{self.create_analysis_path()}/datasets")
#         return read_datasets(path_file)

    def set_models(self, models, extend=True):
        """Set models on datasets.
        Adds `FoVBackgroundModel` if not present already

        Parameters
        ----------
        models : `~gammapy.modeling.models.Models` or str
            Models object or YAML models string
        extend : bool
            Extend the exiting models on the datasets or replace them.
        """
        if not self.datasets or len(self.datasets) == 0:
            raise RuntimeError("Missing datasets")

        log.info("Reading model.")
        if isinstance(models, str):
            models = Models.from_yaml(models)
        elif isinstance(models, Models):
            pass
        elif isinstance(models, DatasetModels) or isinstance(models, list):
            models = Models(models)
        else:
            raise TypeError(f"Invalid type: {models!r}")

        if extend:
            models.extend(self.datasets.models)

        self.datasets.models = models

        bkg_models = []
        for dataset in self.datasets:
            if dataset.tag == "MapDataset" and dataset.background_model is None:
                bkg_models.append(FoVBackgroundModel(dataset_name=dataset.name))
        if bkg_models:
            models.extend(bkg_models)
            self.datasets.models = models

        log.info(models)

    def read_models(self, path, extend=True):
        """Read models from YAML file.

        Parameters
        ----------
        path : str
            path to the model file
        extend : bool
            Extend the exiting models on the datasets or replace them.
        """

        path = make_path(path)
        models = Models.read(path)
        self.set_models(models, extend=extend)
        log.info(f"Models loaded from {path}.")

    def write_models(self, overwrite=True, write_covariance=True):
        """Write models to YAML file.
        File name is taken from the configuration file.
        """

        filename_models = self.config.general.models_file
        if filename_models is not None:
            self.models.write(
                filename_models, overwrite=overwrite, write_covariance=write_covariance
            )
            log.info(f"Models loaded from {filename_models}.")
        else:
            raise RuntimeError("Missing models_file in config.general")

    def read_datasets(self):
        """Read datasets from YAML file.
        File names are taken from the configuration file.

        """

        filename = self.config.general.datasets_file
        filename_models = self.config.general.models_file
        if filename is not None:
            self.datasets = Datasets.read(filename)
            log.info(f"Datasets loaded from {filename}.")
            if filename_models is not None:
                self.read_models(filename_models, extend=False)
        else:
            raise RuntimeError("Missing datasets_file in config.general")

    def write_datasets(self, overwrite=True, write_covariance=True):
        """Write datasets to YAML file.
        File names are taken from the configuration file.

        Parameters
        ----------
        overwrite : bool
            overwrite datasets FITS files
        write_covariance : bool
            save covariance or not
        """

        filename = self.config.general.datasets_file
        filename_models = self.config.general.models_file
        if filename is not None:
            self.datasets.write(
                filename,
                filename_models,
                overwrite=overwrite,
                write_covariance=write_covariance,
            )
            log.info(f"Datasets stored to {filename}.")
            log.info(f"Datasets stored to {filename_models}.")
        else:
            raise RuntimeError("Missing datasets_file in config.general")



In [104]:
from feupy.catalog.pulsar.atnf import SourceCatalogATNF

catalog = SourceCatalogATNF()
source = catalog['PSR J1826-1256']
name = source.name
pos_ra = source.position.ra
pos_dec = source.position.dec
from gammapy.modeling.models import SkyModel
from gammapy.modeling.models import ExpCutoffPowerLawSpectralModel
model = SkyModel(spectral_model=ExpCutoffPowerLawSpectralModel(), name=name)
print(model)
target = Target(name, pos_ra, pos_dec, spectral_model=model.spectral_model)
config= CounterpartsConfig()
config.roi.target = target.dict
config.roi.region_radius = 1*u.deg 
config.roi.region_radius
e_edges_min=0.1*u.TeV 
# e_edges_max=100.*u.TeV
config.energy_range.min = e_edges_min
config.energy_range.max = e_edges_max
config.write("config.yaml", overwrite=True)


SkyModel

  Name                      : PSR J1826-1256
  Datasets names            : None
  Spectral model type       : ExpCutoffPowerLawSpectralModel
  Spatial  model type       : 
  Temporal model type       : 
  Parameters:
    index                         :      1.500   +/-    0.00             
    amplitude                     :   1.00e-12   +/- 0.0e+00 1 / (TeV s cm2)
    reference             (frozen):      1.000       TeV         
    lambda_                       :      0.100   +/-    0.00 1 / TeV     
    alpha                 (frozen):      1.000                   




In [105]:
config = CounterpartsConfig.read("config.yaml")
print(config)

CounterpartsConfig

    general:
        log: {level: info, filename: null, filemode: null, format: null, datefmt: null}
        outdir: .
        n_jobs: 1
        datasets_file: null
        models_file: null
    roi:
        target:
            name: PSR J1826-1256
            position: {frame: icrs, lon: 276.53554166666663 deg, lat: -12.9425 deg}
            model:
                name: PSR J1826-1256
                type: SkyModel
                spectral:
                    type: ExpCutoffPowerLawSpectralModel
                    parameters:
                    - {name: index, value: 1.5}
                    - {name: amplitude, value: 1.0e-12, unit: TeV-1 s-1 cm-2}
                    - {name: reference, value: 1.0, unit: TeV}
                    - {name: lambda_, value: 0.1, unit: TeV-1}
                    - {name: alpha, value: 1.0}
        region_radius: 1.0 deg
    energy_range: {min: 0.1 TeV, max: 100.0 TeV}
    


In [106]:
analysis = Counterparts(config)

Setting logging config: {'level': 'INFO', 'filename': None, 'filemode': None, 'format': None, 'datefmt': None}


In [107]:
analysis.roi

ROI(name = 'PSR J1826-1256', pos_ra = Quantity('276.54deg'),pos_dec = Quantity('-12.94deg'),radius = Quantity('1.00deg'))

In [108]:
analysis.run()

No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.


The error is: (2HWC J1825-134) 'SourceCatalogObject2HWC' object has no attribute 'flux_points'
The error is: (3FGL J1823.2-1339) index -1 is out of bounds for axis 0 with size 0
The error is: (3FGL J1824.5-1351e) index -1 is out of bounds for axis 0 with size 0
The error is: (3FGL J1826.1-1256) index -1 is out of bounds for axis 0 with size 0


No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.


The error is: (3FGL J1829.7-1304) index -1 is out of bounds for axis 0 with size 0


No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point 

The error is: (3HWC J1825-134) 'SourceCatalogObject3HWC' object has no attribute 'flux_points'


No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.


Total number of gamma sources: 20
Total number of flux points tables: 20
Total number of pulsars: 6


In [110]:
analysis.df_roi

,Source name,RA(deg),dec.(deg),Sep.(deg)
0,HESS J1826-130: gamma-cat,276.504181,-13.091110,0.151719
1,HESS J1825-137: gamma-cat,276.554413,-13.580040,0.637805
2,HESS J1826-130: hgps,276.508728,-13.017380,0.079307
3,4FGL J1823.3-1340,275.836304,-13.667600,0.994385
4,4FGL J1824.1-1304,276.032196,-13.072000,0.507240
5,4FGL J1824.4-1350e,276.110992,-13.839000,0.987058
6,4FGL J1826.1-1256,276.535187,-12.941500,0.001058
7,4FGL J1826.5-1202c,276.648590,-12.048400,0.900886
8,4FGL J1828.1-1312,277.027588,-13.201900,0.544988
9,4FGL J1829.4-1256,277.354309,-12.950000,0.797990


In [96]:
energy_range_settings = analysis.config.energy_range
roi_settings = analysis.config.roi
on_region_radius = roi_settings.region_radius
target_settings = roi_settings.target

In [97]:
from gammapy.modeling.models import Model
model = Model.from_dict(target_settings.model)
print(model)

SkyModel

  Name                      : PSR J1826-1256
  Datasets names            : None
  Spectral model type       : ExpCutoffPowerLawSpectralModel
  Spatial  model type       : 
  Temporal model type       : 
  Parameters:
    index                         :      1.500   +/-    0.00             
    amplitude                     :   1.00e-12   +/- 0.0e+00 1 / (TeV s cm2)
    reference             (frozen):      1.000       TeV         
    lambda_                       :      0.100   +/-    0.00 1 / TeV     
    alpha                 (frozen):      1.000                   




In [ ]:
log = logging.getLogger(__name__)

class Simulation:
    """Config-driven high level simulation interface.

    It is initialized by default with a set of configuration parameters and values declared in
    an internal high level interface model, though the user can also provide configuration
    parameters passed as a nested dictionary at the moment of instantiation. In that case these
    parameters will overwrite the default values of those present in the configuration file.

    Parameters
    ----------
    config : dict or `SimulationConfig`
        Configuration options following `SimulationConfig` schema
    """

    def __init__(self, config):
        self.config = config
        self.config.set_logging()
        self.geom = self._create_geometry()
        
        self._obs = self.config.observations
        self._obs_params = self.config.observations.parameters
        self._obs_target = self.config.observations.target
        self._obs_irfs = self.config.observations.irfs
        self._datasets = self.config.datasets

        self.pointing_position = self._create_pointing_position(
            dict_to_skcoord(self._obs_target.position), 
            self._obs_params.offset)
        self.pointing = self._create_pointing(self.pointing_position)

        self._ctao_perf = Irfs
        self._ctao_perf.get_irfs(self._obs_irfs.opt)
        self.observation = self._create_observation(
            dict_to_skcoord(self._obs_target.position), 
            self._obs_params.livetime, 
            self._ctao_perf.irfs, 
            self._ctao_perf.obs_loc
        )
        
#         self.datastore = None
#         self.observations = None
#         self.datasets = None
#         self.fit = Fit()
#         self.fit_result = None
#         self.flux_points = None
#         self.dataset_map = None
#         self.pointing = dict_to_skcoord(self.config.target.position)
#         self._datasets_settings = self.config.datasets
#         self._observations_settings = self.config.observations
#         self._ctao_perf = Irfs
#         self._ctao_perf.get_irfs(self.config.irfs.opt)
#         self.observation = None
#         self.geom = None
#         self.energy_axis_true = self._make_energy_axis(self._datasets_settings.geom.axes.energy_true)
#         self.energy_axis_reco = self._make_energy_axis(self._datasets_settings.geom.axes.energy)
#         self.spectrum_dataset_empty = None
#         self.maker = None
#         self.safe_maker = None
#         self.spectrum_dataset = None
#         self.spectrum_dataset_onoff = None
#         self.wstat = None
        
#     @staticmethod
#     def _create_region_geometry(center, axes):
#         """Create the region geometry."""
#         on_lon = target_position.lon
#         on_lat = target_position.lat
#         frame = target_position.frame
#         pointing = SkyCoord(on_lon, on_lat, frame=frame)
#         self._create_pointing_position(
#             dict_to_skcoord(self._obs_target.position), 
#             self._obs_params.offset)
        
#         on_center = pointing.directional_offset_by(
#             position_angle=pointing.dec, 
#             separation=offset)
#         on_region = CircleSkyRegion(on_center, on_region_settings.radius)
#         return 


    def _create_geometry(self):
        """Create the geometry."""
        log.debug("Creating geometry.")
        geom_settings = self.config.datasets.geom
        obs_settings = self.config.observations
        axes = [self._make_energy_axis(geom_settings.axes.energy)]
        center = dict_to_skcoord(obs_settings.target.position)
        radius = obs_settings.parameters.on_region_radius
        region = self._create_on_region(center, radius)
        return RegionGeom.create(region=region, axes=axes)
    
    @staticmethod
    def _create_on_region(center, radius):
        """Create the region geometry.
        on_region_radius :Angle()
        """
        return CircleSkyRegion(
            center=center, 
            radius=radius
        )
    

    @staticmethod
    def _create_pointing_position(position, separation, position_angle = 0 * u.deg):
        return position.directional_offset_by(position_angle, separation)
    
    @staticmethod
    def _create_pointing(pointing_position):
        """Create pointing."""
        return FixedPointingInfo(
            mode=PointingMode.POINTING,
            fixed_icrs=pointing_position.icrs,
        )
    
    @staticmethod
    def _create_observation(pointing, livetime, irfs, location):
        """Create an observation."""
        return Observation.create(
            pointing=pointing,
            livetime=livetime,
            irfs=irfs,
            location=location,
        )
    
    @property
    def config(self):
        """Simulation configuration (`SimulationConfig`)"""
        return self._config

    @config.setter
    def config(self, value):
        if isinstance(value, dict):
            self._config = SimulationConfig(**value)
        elif isinstance(value, SimulationConfig):
            self._config = value
        else:
            raise TypeError("config must be dict or SimulationConfig.")

    @property
    def models(self):
        if not self.datasets:
            raise RuntimeError("No datasets defined. Impossible to set models.")
        return self.datasets.models

    @models.setter
    def models(self, models):
        self.set_models(models, extend=False)
        
        
    @staticmethod
    def _make_energy_axis(config_axis_energy, per_decade=True):
        return MapAxis.from_energy_bounds(        
            energy_min=config_axis_energy.min, 
            energy_max=config_axis_energy.max, 
            nbin=config_axis_energy.nbins, 
            per_decade=per_decade, 
            name=config_axis_energy.name,
            )